<a href="https://www.kaggle.com/code/jessicanbm/kids-story-template-generator?scriptVersionId=235170445" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Late Elementary School Story Assisstant
## Kaggle Gen AI Intensive Course Capstone Project
### Motivation
I selected this project as my children often had to fill out story templates and outlines for class and felt stuck when looking at a piece of paper.   I often would ask them questions regarding the general plot and then follow up questions to help them fill out the template.   I thought this would be a great use case for students to use the tool as a way to help them be more self sufficient in capturing their thoughts. 

This Notebook Creates a GPT that acts as a thoughtful and professional editor to help students who are feeling "stuck" create detailed story outlines.  Aimed at late elemetary school and young adult readers, including tweens and early teens. It provides feedback on story outlines, helping writers expand, clarify, and refine their narratives while ensuring the content remains age-appropriate. It is sensitive to themes and language suitable for a younger audience and helps authors steer away from overly explicit or mature content.

Story Outline Helper first focuses on helping writers establish a solid story structure by asking for some basic story information then fills in the story outline template with the information and age appropriate detail expansion. 

It avoids all graphic sexual depictions and explicit content. However, it can explore emotional themes like love or platonic feelings as they naturally arise in older children's storytelling, always keeping the tone appropriate and respectful for a younger  audience.

Story Outline Helper speaks in a friendly, encouraging, and supportive tone to make young writers feel comfortable and inspired as they share and grow their stories.

It then creates a JSON based story outline and character list. 

#INPUTS
It reads in a character development template and story template PDF files and uses that as the template for the output response. 

#Outputs
    JSON text with detailed information for all parts of the story outline filled in
    JSON text with all details of the character development templated filled in.


# **Gen AI Capabilities**

# Structured Output / JSON Mode / Controlled Generation
    * The code prompts the Gemini model to generate story outlines and character profiles in JSON format.

# Document Understanding
    * The code extracts text from PDF templates using PyPDF2, enabling the model to understand and utilize the structure of story outlines and character development templates.

# Retrieval-Augmented Generation (RAG)
    * By storing templates in ChromaDB and retrieving them during prompt construction, the code employs RAG to provide the model with relevant context.

# Vector Search / Vector Store / Vector Database
   * ChromaDB serves as a vector store, allowing for efficient retrieval of template embeddings to inform the model's responses.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/short-story-templates/Character_development_template.pdf
/kaggle/input/short-story-templates/Story_outline_template.pdf


In [2]:
# 📦 Install required packages
!pip uninstall -qqy jupyterlab
!pip install -U -q "google-genai==1.7.0" "chromadb==0.6.3" "PyPDF2"

# 🔑 Imports and setup
from google import genai
from kaggle_secrets import UserSecretsClient
from PyPDF2 import PdfReader
import chromadb
from chromadb.config import Settings
from IPython.display import display, clear_output
import ipywidgets as widgets
import json
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 87.5 MB/s eta

In [3]:


# 🔐 Initialize Gemini client
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

# 📄 Load template PDFs
def extract_text_from_pdf(path):
    with open(path, "rb") as f:
        return "\n".join([p.extract_text() for p in PdfReader(f).pages if p.extract_text()])

outline_template = extract_text_from_pdf("/kaggle/input/short-story-templates/Story_outline_template.pdf")
character_template = extract_text_from_pdf("/kaggle/input/short-story-templates/Character_development_template.pdf")

# 🧠 Initialize ChromaDB for RAG
chroma_client = chromadb.PersistentClient(path=".chroma", settings=Settings())
collection = chroma_client.get_or_create_collection("story_outline_templates")
collection.add(documents=[outline_template, character_template], ids=["outline", "characters"])

# 🧑‍🎓 User Input UI
story_idea_widget = widgets.Textarea(placeholder="Enter your story idea (1-2 sentences)...", description='Story Idea:', layout=widgets.Layout(width='100%'))
main_character_widget = widgets.Text(placeholder="Name + a few details...", description='Main Char:', layout=widgets.Layout(width='100%'))
genre_widget = widgets.Text(placeholder="e.g., fantasy, sci-fi, adventure...", description='Genre:', layout=widgets.Layout(width='100%'))
new_info_widget = widgets.Textarea(placeholder="Anything else you want to see?", description='Extra Info:', layout=widgets.Layout(width='100%'))
submit_button = widgets.Button(description="Submit ✨")

# Show UI
print("Hi! Let's create your story outline. ✨")
display(story_idea_widget, main_character_widget, genre_widget, new_info_widget, submit_button)

def on_submit(b):
    clear_output()
    story_idea = story_idea_widget.value
    main_character = main_character_widget.value
    genre = genre_widget.value
    new_info = new_info_widget.value

    print("Thanks! Here's what you entered:\n")
    print(f"Story idea: {story_idea}")
    print(f"Main character: {main_character}")
    print(f"Genre: {genre}")
    print(f"Extra info: {new_info}")

    # 🧾 Story outline prompt
    story_prompt = f"""
You are a helpful writing assistant for kids. Use the following outline template:

{outline_template}

Please fill out the full story outline as JSON for this idea:
Story: {story_idea}
Main Character: {main_character}
Genre: {genre}
You can use the information in {new_info} to fill in additional details in the template.

Avoid all graphic sexual depictions and explicit content. However, it can explore emotional themes like love or platonic feelings as they naturally arise in older children's storytelling, always keeping the tone appropriate and respectful for a younger audience.
"""
    response = client.models.generate_content(
    model='gemini-1.5-pro-002',
    contents=story_prompt
)

    
    story_json = response.text

    print("\n📘 **Generated Story Outline:**\n")
    try:
        story_data = json.loads(story_json)
        for key, value in story_data.items():
            print(f"{key}: {value}\n")
    except json.JSONDecodeError:
        print(story_json)

    # 👤 Character profile prompt
    char_prompt = f"""
Based on this story:
{story_idea}
Main character: {main_character}
Additional info: {new_info}

Using the following character development template:
{character_template}

Create a JSON-formatted character profile for the protagonist.

Avoid all graphic sexual depictions and explicit content. However, it can explore emotional themes like love or platonic feelings as they naturally arise in older children's storytelling, always keeping the tone appropriate and respectful for a younger audience.
"""
    
    char_response = client.models.generate_content(
    model='gemini-1.5-pro-002',
    contents=char_prompt
)
    character_json = char_response.text

    print("\n👤 **Generated Character Profile:**\n")
    try:
        character_data = json.loads(character_json)
        for key, value in character_data.items():
            print(f"{key}: {value}\n")
    except json.JSONDecodeError:
        print(character_json)

submit_button.on_click(on_submit)


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 87.6MiB/s]


Hi! Let's create your story outline. ✨


Textarea(value='', description='Story Idea:', layout=Layout(width='100%'), placeholder='Enter your story idea …

Text(value='', description='Main Char:', layout=Layout(width='100%'), placeholder='Name + a few details...')

Text(value='', description='Genre:', layout=Layout(width='100%'), placeholder='e.g., fantasy, sci-fi, adventur…

Textarea(value='', description='Extra Info:', layout=Layout(width='100%'), placeholder='Anything else you want…

Button(description='Submit ✨', style=ButtonStyle())